### ODA112 - Audit BackUp Response - PwC Connect

First, we import all the libraries we're going to work with across the application.

In [47]:
from boxsdk import OAuth2, Client
import numpy
import pandas
import re
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import tkinter as tk
import tkinter.font as font
from tkinter import ttk

Then we log into our personal BOX account, through the ```Client ID```, ```Client Secret``` and ```Developer Token```. All of this variables are picked up from the application that was created in the BOX Developer Console.

In [63]:
authentication_method = OAuth2(client_id = '0voy64ajgdt0rxxibmeib0elopx7pufu', 
                               client_secret = 'S1mRvfAhutoGs4CTr8ffIMeQeRgEYJpg', 
                               access_token = 'q5634gw47tSMI34oONLUAN9hA3rFF8hF')

my_client = Client(authentication_method)

Then we obtain the Authentication URL for the BOX folder we've created in our personal BOX repository.

In [64]:
box_folder_link = 'https://app.box.com/folder/140701906674'
authentication_url, csrf_token = authentication_method.get_authorization_url(box_folder_link)
print(authentication_url)

https://account.box.com/api/oauth2/authorize?state=box_csrf_token_3w7UIXfQOfhoJRwJ&response_type=code&client_id=0voy64ajgdt0rxxibmeib0elopx7pufu&redirect_uri=https%3A%2F%2Fapp.box.com%2Ffolder%2F140701906674


Interacting with Selenium, we open a new browser session - in this case with Google Chrome application - and refer to the Authentication URL. Through this course of action, we can automatically obtain the code that will be needed to retrieve the ```Access Token``` in the following step.  

In [65]:
chrome = webdriver.Chrome(r"C:\Users\BrunoSecchiari\Desktop\ODA112 - Audit Backup Response - PwC Connect\Chrome Drive\chromedriver.exe")

chrome.get(authentication_url)

login = chrome.find_element_by_name("login")
login.send_keys("brunosecchiari95@gmail.com")
password = chrome.find_element_by_name("password")
password.send_keys("vwjpnz95")
login_submit_button = chrome.find_element_by_name("login_submit")
login_submit_button.click()
consent_accept_button = chrome.find_element_by_name("consent_accept")
consent_accept_button.click()

login = chrome.find_element_by_name("login")
login.send_keys("brunosecchiari95@gmail.com")
login_submit_button = chrome.find_element_by_id("login-submit")
login_submit_button.click()
password = chrome.find_element_by_name("password")
password.send_keys("vwjpnz95")
password_submit_button = chrome.find_element_by_id("login-submit-password")
password_submit_button.click()

my_string = chrome.current_url
position1 = re.search('code=', my_string)
starting_position = position1.span()
position2 = re.search('&state=', my_string)
ending_position = position2.span()
my_finalized_string = my_string[starting_position[1]:ending_position[0]]
print(my_finalized_string)

chrome.close()

kTGcR59hvdxigPukb0lU4rnACDoCCmvr


We exchange the code that was part of the Authentication URL for the ```Access Token```, meaning we're now able to interact with BOX API.

In [66]:
authentication_code = my_finalized_string
access_token, refresh_token = authentication_method.authenticate(authentication_code)

Finally, we'll create all the functions that we're going to work with along the application.

##### Retrieve File ID Function

In [6]:
def get_file_id(client, file_name, folder_id):
    
    file_id = None
    
    file_name_with_extension = file_name.split('.')
    file_name = file_name_with_extension[0]
    file_extension = file_name_with_extension[1]
    
    my_list = my_client.search().query(query = f'"{file_name}"', extension = [f'{file_extension}'], limit = 1)
    
    for i in my_list:
        file_id = i.id
    if not file_id:
        raise Exception(f'File was not found in Folder ID: {folder_id}.')
    return file_id

In [36]:
my_file_id = get_file_id(my_client, 'List of Entity - Batch 1.xlsx', '140701906674')
my_file_id

'844872746386'

##### Create Shared Link Function

In [67]:
def create_shared_link(client, file_id):
    
    my_shared_link = None
    
    try:
        my_shared_link = client.file(file_id).get_shared_link(access = 'open', allow_download = True)
        print(f"Shared Link was successfully created for File ID '{file_id}'.")
        return my_shared_link
    
    except BoxAPIException as Error:
        raise Exception(f"An error ocurred while trying to create the Shared Link for File ID: '{file_id}'.")

In [38]:
create_shared_link(my_client, my_file_id)

Shared Link was successfully created for File ID '844872746386'.


'https://app.box.com/s/6il3yk7dxi7dadzaynezam2jhndv8lle'

##### Retrieve Shared Link Function

In [68]:
def get_shared_link(client, file_id):
    
    my_shared_link = None
    
    try:
        my_shared_link = client.file(file_id).get().shared_link
        print(f"The Shared Link for File ID '{file_id}' is the following: ")
        return my_shared_link['download_url']
    
    except BoxAPIException as Error:
        raise Exception(f"An error ocurred while trying to obtain the Shared Link for File ID '{file_id}'.")

In [40]:
get_shared_link(my_client, my_file_id)

The Shared Link for File ID '844872746386' is the following: 


'https://app.box.com/shared/static/6il3yk7dxi7dadzaynezam2jhndv8lle.xlsx'

##### Upload Function

In [69]:
def upload_file(client, folder_id, file_path):
    
    file_for_upload = None
    
    try:
        file_for_upload = client.folder(folder_id).upload(file_path)
        print(f"File was successfully uploaded to BOX Folder '{folder_id}' with the following File ID '{file_for_upload.id}'.")
    
    except BoxAPIException as Error:
        raise Exception(f"An error ocurred while trying to upload the file to BOX Folder '{folder_id}'.")

In [26]:
upload_file(my_client, '140701906674', 
            r"C:\Users\BrunoSecchiari\Desktop\ODA112 - Audit Backup Response - PwC Connect\List of Entity - Batch 1.xlsx")

File was successfully uploaded to BOX Folder '140701906674' with the following File ID '841995560519'.


##### Download Function

In [70]:
def download_file(client, file_id, file_path):
    
    download_path = None
    
    try:
        download_path = open(file_path, 'wb')
        client.file(file_id).download_to(download_path)
        print(f"File ID '{file_id}' was successfully downloaded.")
        
    except BoxAPIException as Error:
        raise Exception(f"An error ocurred while trying to download File ID '{file_id}'.")

In [20]:
download_file(my_client, my_file_id, r"C:\Users\BrunoSecchiari\Desktop\List of Entity - Batch 1.xlsx")

File ID '841995560519' was successfully downloaded.


### Step By Step

In [71]:
list_of_entity_file_id = get_file_id(my_client, 'List of Entity - Batch 1.xlsx', '143056046187')
list_of_entity_file_id

'844872746386'

In [72]:
download_file(my_client, list_of_entity_file_id, r"C:\Users\BrunoSecchiari\Desktop\List of Entity - Batch 1.xlsx")

File ID '844872746386' was successfully downloaded.


In [73]:
list_of_entity = pandas.read_excel(r"C:\Users\BrunoSecchiari\Desktop\List of Entity - Batch 1.xlsx",
                                              sheet_name = 'Audit 2019',
                                              headers = 0)
list_of_entity.head(3)

,Entity Name as per PwC Connect,Entity Name as per IBM BOX,Entity backup BOX Link
0,IBM (China) Investment Company Limited,IBM (China) Investment Company Limited,https://ibm.ent.box.com/folder/127121561693
1,IBM - General CAS/Info,IBM - General CAS & Info,https://ibm.ent.box.com/folder/129034587915
2,IBM A/NZ Holdings,IBM ANZ Holdings,https://ibm.ent.box.com/folder/126302246731


In [74]:
print(list_of_entity.isnull().sum())
print()

list_of_entity = list_of_entity.dropna()
print(list_of_entity.isnull().sum())

Entity Name as per PwC Connect    0
Entity Name as per IBM BOX        3
Entity backup BOX Link            3
dtype: int64

Entity Name as per PwC Connect    0
Entity Name as per IBM BOX        0
Entity backup BOX Link            0
dtype: int64


In [75]:
def get_backup_box_link(dataframe, serie):
    
    backup_box_link = []
    
    try:      
        for i in dataframe.loc[:, serie]:
            backup_box_link.append(i)
        return backup_box_link
    
    except:
        raise Exception(f"An error ocurred while trying to parse through the DataFrame '{dataframe}'.")

In [76]:
backup_box_link = get_backup_box_link(list_of_entity, 'Entity backup BOX Link')
backup_box_link

['https://ibm.ent.box.com/folder/127121561693',
 'https://ibm.ent.box.com/folder/129034587915',
 'https://ibm.ent.box.com/folder/126302246731',
 'https://ibm.ent.box.com/folder/123060631068',
 'https://ibm.ent.box.com/folder/126095157878',
 'https://ibm.ent.box.com/folder/119755987746',
 'https://ibm.ent.box.com/folder/121468561047',
 'https://ibm.ent.box.com/folder/130526260244',
 'https://ibm.ent.box.com/folder/118002107225',
 'https://ibm.ent.box.com/folder/123246726397',
 'https://ibm.ent.box.com/folder/125069499764',
 'https://ibm.ent.box.com/folder/126473281134',
 'https://ibm.ent.box.com/folder/126472252124',
 'https://ibm.ent.box.com/folder/119754677723',
 'https://ibm.ent.box.com/folder/121471907428',
 'https://ibm.ent.box.com/folder/121473104525',
 'https://ibm.ent.box.com/folder/126473097672',
 'https://ibm.ent.box.com/folder/121475746460',
 'https://ibm.ent.box.com/folder/126293419252',
 'https://ibm.ent.box.com/folder/118494305662',
 'https://ibm.ent.box.com/folder/1271713

In [77]:
backup_box_folder = []

for i in range(len(backup_box_link)):
    j = backup_box_link[i].split('/')[4]
    backup_box_folder.append(j)
    
backup_box_folder

['127121561693',
 '129034587915',
 '126302246731',
 '123060631068',
 '126095157878',
 '119755987746',
 '121468561047',
 '130526260244',
 '118002107225',
 '123246726397',
 '125069499764',
 '126473281134',
 '126472252124',
 '119754677723',
 '121471907428',
 '121473104525',
 '126473097672',
 '121475746460',
 '126293419252',
 '118494305662',
 '127171339369',
 '126732205164',
 '121154461798',
 '126444421704',
 '126690635559',
 '126305279988',
 '126002926074',
 '126683991839',
 '119510287216',
 '118065281660',
 '125494399662',
 '130531964825',
 '125059250472',
 '119284855199',
 '121032215319',
 '126507758355',
 '119766987475',
 '126448365689',
 '126763744887',
 '124475856239?s=qlr01i1f2g8s5xi77woqgj18bvgykdxw',
 '129036871389',
 '119492559429',
 '129037781161',
 '129038255441',
 '126736499766',
 '126673329694',
 '126623584560',
 '120236770366',
 '117926649753',
 '123060631068',
 '119755987746',
 '129118164434',
 '126896637205',
 '126913994195',
 '119509230985',
 '126806717697',
 '12680514995

In [78]:
my_file_id = get_file_id(my_client, 
                         'IBM World Trade Corporation - Brunei Darussalam Operations FY2019 - Connect Request Details - 06.Aug.2020.xlsx', 
                         '143055972076')
my_file_id

'844875688650'

In [79]:
download_file(my_client, 
              my_file_id, 
              r'C:\Users\BrunoSecchiari\Desktop\IBM World Trade Corporation - Brunei Darussalam Operations FY2019 - Connect Request Details - 06.Aug.2020.xlsx')

File ID '844875688650' was successfully downloaded.


In [80]:
brunei_darussalam = pandas.read_excel(r'C:\Users\BrunoSecchiari\Desktop\IBM World Trade Corporation - Brunei Darussalam Operations FY2019 - Connect Request Details - 06.Aug.2020.xlsx', 
                                      sheet_name = 'Requests', 
                                      skiprows = 6)
brunei_darussalam.head(3)

,Engagement,Request ID,Type of testing,Interim/Final,FSLI,Request,Matching Criterion with BOX,Description,Status,Due Date,...,Secondary Owners,Primary Requestee,Secondary Requestees,"Access Restricted to PwC Admins, Owners & Requestees Only?",Reassignment Restricted?,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Hyperlink to CAS Response
0,Process A - Services Accounting,11,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 11(June'19) (Invoice number...,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...,Kindly provide the following items:\n1) Advise...,Accepted,2019-08-14,...,NaN,Nurfatin Shuhaimi,"Erica Nyssa T Jose, Robert Adrian T Cruz, Mari...",No,No,No,NaN,"BR0671+-+SSpremium+-+Bank+Islam+Brunei.pdf, IR...","Item #s 1, 3 & 7",https://ibm.ent.box.com/folder/121095780672
1,Process A - Services Accounting,10,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 10(May'19) (Invoice number:...,10 - SVS Tod TSS Sample 10(May'19) (Invoice nu...,Kindly provide the following items:\n1) Advise...,Accepted,2019-10-04,...,NaN,Nurfatin Shuhaimi,"Erica Nyssa T Jose, Robert Adrian T Cruz, Mari...",No,No,No,NaN,The billing for Contract BR0645 BR0650 BR0651....,"Item #s 1, 3 & 7, Amount in invoice #5U1191 in...",https://ibm.ent.box.com/folder/121095775426
2,Process A - Services Accounting,9,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 9(May'19) (Invoice number:5...,9 - SVS Tod TSS Sample 9(May'19) (Invoice numb...,Kindly provide the following items:\n1) Advise...,Accepted,2019-10-22,...,NaN,Afeeqa Aiyeesha Asli,Nurfahana Abu Bakar,No,No,No,NaN,"5A1295.xlsx, 9 (643,02) TSS Accountant Respons...","Kindly provide signed contract #BR0645, Revenu...",https://ibm.ent.box.com/folder/121092592841


In [81]:
brunei_darussalam['Matching Criterion with BOX - Updated'] = brunei_darussalam.apply(lambda x: str(x['Request ID']) + ' - ' + \
                                                                           x['Request'], axis = 1)
brunei_darussalam.head(3)

,Engagement,Request ID,Type of testing,Interim/Final,FSLI,Request,Matching Criterion with BOX,Description,Status,Due Date,...,Primary Requestee,Secondary Requestees,"Access Restricted to PwC Admins, Owners & Requestees Only?",Reassignment Restricted?,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Hyperlink to CAS Response,Matching Criterion with BOX - Updated
0,Process A - Services Accounting,11,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 11(June'19) (Invoice number...,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...,Kindly provide the following items:\n1) Advise...,Accepted,2019-08-14,...,Nurfatin Shuhaimi,"Erica Nyssa T Jose, Robert Adrian T Cruz, Mari...",No,No,No,NaN,"BR0671+-+SSpremium+-+Bank+Islam+Brunei.pdf, IR...","Item #s 1, 3 & 7",https://ibm.ent.box.com/folder/121095780672,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...
1,Process A - Services Accounting,10,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 10(May'19) (Invoice number:...,10 - SVS Tod TSS Sample 10(May'19) (Invoice nu...,Kindly provide the following items:\n1) Advise...,Accepted,2019-10-04,...,Nurfatin Shuhaimi,"Erica Nyssa T Jose, Robert Adrian T Cruz, Mari...",No,No,No,NaN,The billing for Contract BR0645 BR0650 BR0651....,"Item #s 1, 3 & 7, Amount in invoice #5U1191 in...",https://ibm.ent.box.com/folder/121095775426,10 - SVS Tod TSS Sample 10(May'19) (Invoice nu...
2,Process A - Services Accounting,9,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 9(May'19) (Invoice number:5...,9 - SVS Tod TSS Sample 9(May'19) (Invoice numb...,Kindly provide the following items:\n1) Advise...,Accepted,2019-10-22,...,Afeeqa Aiyeesha Asli,Nurfahana Abu Bakar,No,No,No,NaN,"5A1295.xlsx, 9 (643,02) TSS Accountant Respons...","Kindly provide signed contract #BR0645, Revenu...",https://ibm.ent.box.com/folder/121092592841,9 - SVS Tod TSS Sample 9(May'19) (Invoice numb...


In [82]:
brunei_darussalam['Matching Criterion with BOX - Finalized'] = brunei_darussalam['Matching Criterion with BOX - Updated']
brunei_darussalam.head(3)

,Engagement,Request ID,Type of testing,Interim/Final,FSLI,Request,Matching Criterion with BOX,Description,Status,Due Date,...,Secondary Requestees,"Access Restricted to PwC Admins, Owners & Requestees Only?",Reassignment Restricted?,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Hyperlink to CAS Response,Matching Criterion with BOX - Updated,Matching Criterion with BOX - Finalized
0,Process A - Services Accounting,11,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 11(June'19) (Invoice number...,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...,Kindly provide the following items:\n1) Advise...,Accepted,2019-08-14,...,"Erica Nyssa T Jose, Robert Adrian T Cruz, Mari...",No,No,No,NaN,"BR0671+-+SSpremium+-+Bank+Islam+Brunei.pdf, IR...","Item #s 1, 3 & 7",https://ibm.ent.box.com/folder/121095780672,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...
1,Process A - Services Accounting,10,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 10(May'19) (Invoice number:...,10 - SVS Tod TSS Sample 10(May'19) (Invoice nu...,Kindly provide the following items:\n1) Advise...,Accepted,2019-10-04,...,"Erica Nyssa T Jose, Robert Adrian T Cruz, Mari...",No,No,No,NaN,The billing for Contract BR0645 BR0650 BR0651....,"Item #s 1, 3 & 7, Amount in invoice #5U1191 in...",https://ibm.ent.box.com/folder/121095775426,10 - SVS Tod TSS Sample 10(May'19) (Invoice nu...,10 - SVS Tod TSS Sample 10(May'19) (Invoice nu...
2,Process A - Services Accounting,9,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 9(May'19) (Invoice number:5...,9 - SVS Tod TSS Sample 9(May'19) (Invoice numb...,Kindly provide the following items:\n1) Advise...,Accepted,2019-10-22,...,Nurfahana Abu Bakar,No,No,No,NaN,"5A1295.xlsx, 9 (643,02) TSS Accountant Respons...","Kindly provide signed contract #BR0645, Revenu...",https://ibm.ent.box.com/folder/121092592841,9 - SVS Tod TSS Sample 9(May'19) (Invoice numb...,9 - SVS Tod TSS Sample 9(May'19) (Invoice numb...


In [83]:
character = ['<', '>', ':', '"', '/', '|', '\?', '\*']

for i in character:
    brunei_darussalam['Matching Criterion with BOX - Finalized'] = \
    brunei_darussalam['Matching Criterion with BOX - Finalized'].replace(i, '', regex = True)

In [84]:
print(brunei_darussalam['Matching Criterion with BOX - Updated'][0])
print('-----VERSUS-----')
print(brunei_darussalam['Matching Criterion with BOX - Finalized'][0])

11 - SVS Tod TSS Sample 11(June'19) (Invoice number:5A1300)
-----VERSUS-----
11 - SVS Tod TSS Sample 11(June'19) (Invoice number5A1300)


In [85]:
print(brunei_darussalam['Matching Criterion with BOX - Updated'][14])
print('-----VERSUS-----')
print(brunei_darussalam['Matching Criterion with BOX - Finalized'][14])

18 - Request ES Number/ Supporting Documents of Selected Contracts Q1
-----VERSUS-----
18 - Request ES Number Supporting Documents of Selected Contracts Q1


In [86]:
brunei_darussalam['Matching Criterion with BOX - Finalized'] = \
brunei_darussalam['Matching Criterion with BOX - Finalized'].apply(lambda x: x[0:54])

In [87]:
print(brunei_darussalam['Matching Criterion with BOX - Updated'][0])
print('-----VERSUS-----')
print(brunei_darussalam['Matching Criterion with BOX - Finalized'][0])

11 - SVS Tod TSS Sample 11(June'19) (Invoice number:5A1300)
-----VERSUS-----
11 - SVS Tod TSS Sample 11(June'19) (Invoice number5A1


In [88]:
print(brunei_darussalam['Matching Criterion with BOX - Updated'][14])
print('-----VERSUS-----')
print(brunei_darussalam['Matching Criterion with BOX - Finalized'][14])

18 - Request ES Number/ Supporting Documents of Selected Contracts Q1
-----VERSUS-----
18 - Request ES Number Supporting Documents of Selecte


In [89]:
box_folder_dictionary = {}
box_folder = my_client.folder(folder_id = '143056160663').get_items()

for i in box_folder:
    box_folder_dictionary.update({i.name: i.id})
    
box_folder_dictionary

{"11 - SVS Tod TSS Sample 11(June'19) (Invoice number5A1": '143133022750',
 '6 - STG TOD Samples (Nov) - Summary (Geodis) Upon Amor': '143577220423',
 '66 - SVS ToD Others Samples Dec - Summary (STS)': '143577350295',
 '75 - Unpredictability -SWG Feb 2020 Samples (Geodis Sum': '143577359852',
 '96 - SVS TSS Jan Cut-Off Top Up Samples - STS Summary': '143576750833'}

In [90]:
box_folder_dictionary[brunei_darussalam['Matching Criterion with BOX - Finalized'][0]]

'143133022750'

In [91]:
box_folder_dictionary_key = []

for i in box_folder_dictionary.keys():
    box_folder_dictionary_key.append(i)
    
box_folder_dictionary_value = []

for i in box_folder_dictionary.values():
    box_folder_dictionary_value.append(i)    

box_folder = pandas.DataFrame({'Matching Criterion with BOX - Finalized': box_folder_dictionary_key, 
                                         'BOX Folder ID': box_folder_dictionary_value})
box_folder

,Matching Criterion with BOX - Finalized,BOX Folder ID
0,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...,143133022750
1,6 - STG TOD Samples (Nov) - Summary (Geodis) U...,143577220423
2,66 - SVS ToD Others Samples Dec - Summary (STS),143577350295
3,75 - Unpredictability -SWG Feb 2020 Samples (G...,143577359852
4,96 - SVS TSS Jan Cut-Off Top Up Samples - STS ...,143576750833


In [92]:
box_folder['BOX Folder ID - Format'] = box_folder['BOX Folder ID'].apply(lambda x: 'https://ibm.ent.box.com/folder/' + str(x))
box_folder

,Matching Criterion with BOX - Finalized,BOX Folder ID,BOX Folder ID - Format
0,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...,143133022750,https://ibm.ent.box.com/folder/143133022750
1,6 - STG TOD Samples (Nov) - Summary (Geodis) U...,143577220423,https://ibm.ent.box.com/folder/143577220423
2,66 - SVS ToD Others Samples Dec - Summary (STS),143577350295,https://ibm.ent.box.com/folder/143577350295
3,75 - Unpredictability -SWG Feb 2020 Samples (G...,143577359852,https://ibm.ent.box.com/folder/143577359852
4,96 - SVS TSS Jan Cut-Off Top Up Samples - STS ...,143576750833,https://ibm.ent.box.com/folder/143576750833


In [93]:
brunei_darussalam = brunei_darussalam.merge(box_folder, 
                                            how = 'left', 
                                            on = 'Matching Criterion with BOX - Finalized')
brunei_darussalam.head(3)

,Engagement,Request ID,Type of testing,Interim/Final,FSLI,Request,Matching Criterion with BOX,Description,Status,Due Date,...,Reassignment Restricted?,Response Document Download Restricted?,Template Documents,Response Documents,Discussions,Hyperlink to CAS Response,Matching Criterion with BOX - Updated,Matching Criterion with BOX - Finalized,BOX Folder ID,BOX Folder ID - Format
0,Process A - Services Accounting,11,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 11(June'19) (Invoice number...,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...,Kindly provide the following items:\n1) Advise...,Accepted,2019-08-14,...,No,No,NaN,"BR0671+-+SSpremium+-+Bank+Islam+Brunei.pdf, IR...","Item #s 1, 3 & 7",https://ibm.ent.box.com/folder/121095780672,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...,11 - SVS Tod TSS Sample 11(June'19) (Invoice n...,143133022750,https://ibm.ent.box.com/folder/143133022750
1,Process A - Services Accounting,10,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 10(May'19) (Invoice number:...,10 - SVS Tod TSS Sample 10(May'19) (Invoice nu...,Kindly provide the following items:\n1) Advise...,Accepted,2019-10-04,...,No,No,NaN,The billing for Contract BR0645 BR0650 BR0651....,"Item #s 1, 3 & 7, Amount in invoice #5U1191 in...",https://ibm.ent.box.com/folder/121095775426,10 - SVS Tod TSS Sample 10(May'19) (Invoice nu...,10 - SVS Tod TSS Sample 10(May'19) (Invoice nu...,NaN,NaN
2,Process A - Services Accounting,9,Substantive Testing,Interim,NaN,SVS Tod TSS Sample 9(May'19) (Invoice number:5...,9 - SVS Tod TSS Sample 9(May'19) (Invoice numb...,Kindly provide the following items:\n1) Advise...,Accepted,2019-10-22,...,No,No,NaN,"5A1295.xlsx, 9 (643,02) TSS Accountant Respons...","Kindly provide signed contract #BR0645, Revenu...",https://ibm.ent.box.com/folder/121092592841,9 - SVS Tod TSS Sample 9(May'19) (Invoice numb...,9 - SVS Tod TSS Sample 9(May'19) (Invoice numb...,NaN,NaN


In [94]:
brunei_darussalam['Matching Criterion with BOX'] = brunei_darussalam['Matching Criterion with BOX - Finalized']
brunei_darussalam['Hyperlink to CAS Response'] = brunei_darussalam['BOX Folder ID - Format']

brunei_darussalam.drop(['Matching Criterion with BOX - Updated', 
                        'Matching Criterion with BOX - Finalized', 
                        'BOX Folder ID', 
                        'BOX Folder ID - Format'], axis = 1, inplace = True)

In [95]:
brunei_darussalam.to_excel(r'C:\Users\BrunoSecchiari\Desktop\Trial&Error.xlsx',
                          sheet_name = 'Requests',
                          index = False)